# Initial Cleaning/ setup procedures


#### _1. Import libraries and set path to pkl file_

In [1]:
import pandas as pd
import os
from datetime import date, timedelta
import datetime
import numpy as np
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt

ROOT_DIR = os.path.realpath(os.path.join(os.getcwd(), '..'))
pkl_loc = os.path.join(ROOT_DIR, 'data', 'data','weatherraw.pkl')
pkl_loc

'C:\\Users\\jesse\\OneDrive\\Documents\\homework\\Mich\\696\\ms2_git\\data\\data\\weatherraw.pkl'

### _2. Create dataframe and run basic eda_

In [2]:
df = pd.read_pickle(pkl_loc)
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 62183652 entries, ('04AEH', Timestamp('2022-04-23 14:00:00')) to ('ZYITU', Timestamp('2022-10-01 23:00:00'))
Data columns (total 11 columns):
 #   Column  Dtype  
---  ------  -----  
 0   temp    float64
 1   dwpt    float64
 2   rhum    float64
 3   prcp    float64
 4   snow    float64
 5   wdir    float64
 6   wspd    float64
 7   wpgt    float64
 8   pres    float64
 9   tsun    float64
 10  coco    float64
dtypes: float64(11)
memory usage: 5.4+ GB


In [3]:
ROOT_DIR = os.path.realpath(os.path.join(os.getcwd(), '..'))
pkl_loc = os.path.join(ROOT_DIR, 'data', 'data','octweatherraw.pkl')
pkl_loc
df2 = pd.read_pickle(pkl_loc)
df2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 830520 entries, ('04AEH', Timestamp('2022-10-01 00:00:00')) to ('ZYITU', Timestamp('2022-10-15 23:00:00'))
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   temp    830520 non-null  float64
 1   dwpt    830520 non-null  float64
 2   rhum    830520 non-null  float64
 3   prcp    830520 non-null  float64
 4   snow    0 non-null       float64
 5   wdir    830520 non-null  float64
 6   wspd    830520 non-null  float64
 7   wpgt    0 non-null       float64
 8   pres    830520 non-null  float64
 9   tsun    0 non-null       float64
 10  coco    830219 non-null  float64
dtypes: float64(11)
memory usage: 73.0+ MB


In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.describe()

In [5]:
df = pd.concat([df, df2])
df.shape


(63014172, 11)

### _3. Remove unneeded columns_


In [6]:
df.drop(columns=['snow', 'wpgt','tsun','coco'],inplace=True)
df.head()

temp  dwpt  rhum  prcp   wdir  wspd    pres
station time                                                            
04AEH   2022-04-23 14:00:00  10.0   0.0  50.0   NaN  200.0   7.0  1030.0
        2022-04-23 15:00:00  10.0  -0.8  47.0   NaN  170.0   6.0  1029.0
        2022-04-23 16:00:00  12.0   0.7  46.0   NaN  170.0   9.0  1028.0
        2022-04-23 17:00:00  13.0  -0.6  39.0   NaN  190.0   6.0  1027.0
        2022-04-23 18:00:00  14.0   0.6  40.0   NaN   80.0   7.0  1026.0

In [7]:
df=df.reset_index()

### _4. Counting Stations_

In [8]:
df.groupby("station").count() #clearly some stations are missing

,time,temp,dwpt,rhum,prcp,wdir,wspd,pres
station,,,,,,,,
04AEH,4234,4234,4234,4234,4208,4230,4230,4234
0CC8G,11000,11000,11000,11000,10997,11000,11000,11000
0CNUO,11027,11027,11027,11027,11024,11027,11027,11027
0CO7B,11027,11027,11027,11027,11024,11027,11027,11027
0FV1F,11027,11027,11027,11027,11024,11027,11027,11027
...,...,...,...,...,...,...,...,...
ZRBBD,23710,23709,23709,23710,23709,23588,23705,23709
ZUQJS,3918,3918,3918,3918,3905,3918,3918,3907
ZWC6W,24478,24477,24477,24477,11032,24470,24478,24477


In [9]:
dfstations=df.groupby("station").count()
df_stationskeep = dfstations[dfstations['time']>5000].reset_index() #creating this to know which stations to keep later

In [10]:
df.time.describe()

C:\Users\jesse\AppData\Local\Temp\ipykernel_3860\1129510735.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.time.describe()


count                63014172
unique                  33216
top       2022-10-01 14:00:00
freq                     4614
first     2019-01-01 00:00:00
last      2022-10-15 23:00:00
Name: time, dtype: object

In [11]:
cln_pkl_loc = os.path.join(ROOT_DIR, 'data_cleaning','cleanweather.pkl')
df.to_pickle(cln_pkl_loc)